<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/Data_Ac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing python libraries that will be use to get data from yahoo finance

In [1]:
! pip install yahooquery
! pip install urllib3==1.26
! pip install requests==2.25
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Importing modules that will be used to filter data

In [2]:
from yahooquery import Ticker
from bs4 import BeautifulSoup

import concurrent.futures
import pandas as pd
import requests
import re
import os

Scrapping Data from Infomoney website, getting a list of stocks from IBOVESPA.

In [3]:
url = requests.get('https://www.infomoney.com.br/cotacoes/empresas-b3/')

soup = BeautifulSoup(url.text, 'html.parser')

ticker_name_list = soup.find_all('td', class_ = 'strong')

ticker = str(ticker_name_list)

tickers = re.findall('>.*</',ticker)

tkrs = []

# strnot = ("11","F","34")
for i in tickers:
  if "11" not in i and "F" not in i:
    tkr = i.strip("</>")
    tkr = tkr + ".SA"
    tkrs.append(tkr)

In [4]:
def fetch_historical_financial(tickers):
    # Yahoo finance requisition getting companies ticker
    sumary = Ticker(tickers)

    # getting companies financial data
    data = sumary.all_financial_data()

    df = pd.DataFrame(data)

    return df

In [5]:
def fetch_historical_balance(tickers):
    # Balance Ticker lists
    balance = Ticker(tickers)

    # Getting companies balance data
    data = balance.balance_sheet(trailing=False)

    df = pd.DataFrame(data)

    # df = df.reset_index(drop=True)  # Reset the index of the final DataFrame
    # df['Symbol'] = tickers

    return df

In [6]:
def fetch_historical_finandata(tickers):
    # Balance Ticker lists
    financial = Ticker(tickers)

    # Getting companies balance data
    types = ['symbol', 'TotalDebt', 'TotalAssets', 'EBIT', 'EBITDA', 'PeRatio']
        
    data = financial.get_financial_data(types, trailing=False)

    df = pd.DataFrame(data)

    df = df.reset_index(drop=True)

    return df

In [7]:
def fetch_historical_price(tickers):
    pricetkrs = Ticker(tickers)

    dataprice = pricetkrs.price

    df = pd.DataFrame(dataprice)

    return df.T

In [8]:
# Define a function to fetch data for each batch of tickers
def fetch_data_batch(tkrs,FS):
    if FS == 1:
        data = fetch_historical_financial(tkrs)

    elif FS == 2:
        data = fetch_historical_balance(tkrs)

    elif FS == 3:
        data = fetch_historical_finandata(tkrs)

    elif FS == 4:
        data = fetch_historical_price(tkrs)

    return data

In [9]:
def ElementsList(elements_per_list,tkrs):
    # Divide the ticker list into smaller lists
    smaller_lists = [tkrs[i:i+elements_per_list] for i in range(0, len(tkrs), elements_per_list)]
    
    return smaller_lists

In [10]:
# # # Function that processes operations based on different flags
# # # BS: Batch size
# # # FS: Function Signal
# # # TL: Ticker List

def Process(BS,TL,FS):
    # Lists of elements
    lists = ElementsList(BS,TL)
    
    # dataframe that contains the results of the processed files
    df_final = pd.DataFrame()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the threads for processing
        futures = [executor.submit(fetch_data_batch, list_batch, FS) for list_batch in lists[:BS]]

        # Iterate through the completed threads and concatenate the results
        for future in concurrent.futures.as_completed(futures):
            data = future.result()
            df_final = pd.concat([df_final, data])

    # Reset the index of the final DataFrame
    df_final = df_final.reset_index(drop=True)

    return df_final

In [11]:
historical_financial = Process(20,tkrs,1)

historical_financial

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AllowanceForDoubtfulAccountsReceivable,Amortization,AmortizationCashFlow,...,DepletionIncomeStatement,NotesReceivable,DividendsReceivedDirect,InterestPaidDirect,PaymentsonBehalfofEmployees,PaymentstoSuppliersforGoodsandServices,ReceiptsfromCustomers,TaxesRefundPaidDirect,Depletion,ExcessTaxBenefitFromStockBasedCompensation
0,2019-12-31,12M,BRL,116629000.0,9.853200e+07,-48235000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,12M,BRL,205304000.0,2.201320e+08,-71737000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-31,12M,BRL,445286000.0,1.268770e+08,-103941000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-31,12M,BRL,336048000.0,0.000000e+00,-155715000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-31,12M,BRL,6729000.0,8.468000e+06,-13696000.0,NaN,-397000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,2022-12-31,12M,BRL,111557000.0,3.030059e+09,-93055000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1219,2019-12-31,12M,BRL,167774000.0,2.061882e+09,-38547000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1220,2020-12-31,12M,BRL,153346000.0,2.804373e+09,-54900000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221,2021-12-31,12M,BRL,83666000.0,2.344141e+09,-69655000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
historical_balance = Process(20,tkrs,2)

historical_balance

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AllowanceForDoubtfulAccountsReceivable,AssetsHeldForSaleCurrent,AvailableForSaleSecurities,...,ReceivablesAdjustmentsAllowances,UnrealizedGainLoss,CurrentDeferredAssets,InvestmentsinJointVenturesatCost,CurrentDeferredTaxesAssets,DuefromRelatedPartiesNonCurrent,DuetoRelatedPartiesNonCurrent,LoansReceivable,NonCurrentNoteReceivables,NotesReceivable
0,2019-12-31,12M,BRL,116629000.0,9.853200e+07,-48235000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,12M,BRL,205304000.0,2.201320e+08,-71737000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-31,12M,BRL,445286000.0,1.268770e+08,-103941000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-31,12M,BRL,336048000.0,0.000000e+00,-155715000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-31,12M,BRL,6729000.0,8.468000e+06,-13696000.0,NaN,-397000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,2022-12-31,12M,BRL,111557000.0,3.030059e+09,-93055000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.794693e+09,NaN,NaN,NaN,NaN,NaN,NaN
1217,2019-12-31,12M,BRL,167774000.0,2.061882e+09,-38547000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.198004e+09,NaN,NaN,NaN,NaN,NaN,NaN
1218,2020-12-31,12M,BRL,153346000.0,2.804373e+09,-54900000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.858002e+09,NaN,NaN,NaN,NaN,NaN,NaN
1219,2021-12-31,12M,BRL,83666000.0,2.344141e+09,-69655000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.299479e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
historical_finandata = Process(20,tkrs,3)

historical_finandata

,asOfDate,periodType,currencyCode,EBIT,TotalAssets,TotalDebt
0,2019-12-31,12M,BRL,1.686200e+07,4.020068e+09,1.519503e+09
1,2020-12-31,12M,BRL,-3.148920e+08,3.367654e+09,1.447232e+09
2,2021-12-31,12M,BRL,-4.673300e+07,3.423185e+09,1.462987e+09
3,2022-12-31,12M,BRL,-2.865980e+08,3.464573e+09,1.702852e+09
4,2019-12-31,12M,BRL,1.648471e+09,6.037092e+09,1.587680e+09
...,...,...,...,...,...,...
1216,2022-12-31,12M,BRL,3.346154e+09,3.224355e+10,8.041697e+09
1217,2019-12-31,12M,BRL,2.305742e+09,2.159909e+10,3.293751e+09
1218,2020-12-31,12M,BRL,4.389012e+09,2.459236e+10,4.535732e+09
1219,2021-12-31,12M,BRL,4.174635e+09,2.897674e+10,7.416547e+09


In [14]:
historical_data = Process(20,tkrs,4)

historical_data

,maxAge,regularMarketChangePercent,regularMarketChange,regularMarketTime,priceHint,regularMarketPrice,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,...,underlyingSymbol,shortName,longName,currency,quoteSourceName,currencySymbol,fromCurrency,toCurrency,lastMarket,marketCap
0,1,-0.067834,-0.31,2023-07-12 17:07:42,4,4.26,4.64,4.26,1656600,4.57,...,None,ZAMP S.A. ON NM,Zamp S.A.,BRL,Delayed Quote,R$,None,None,None,1148044544
1,1,-0.071931,-1.33,2023-07-12 14:35:36,2,17.16,17.28,17.08,83,18.49,...,None,CLOUDFLARE DRN,"Cloudflare, Inc.",BRL,Delayed Quote,R$,None,None,None,NaN
2,1,0.024503,1.269997,2023-07-12 17:10:00,2,53.1,53.22,51.42,114347,51.83,...,None,META PLAT DRN,"Meta Platforms, Inc.",BRL,Delayed Quote,R$,None,None,None,3885698383872
3,1,-0.023256,-0.03,2023-07-12 17:07:00,4,1.26,1.32,1.25,4159700,1.29,...,None,AERIS ON NM,Aeris Indústria e Comércio de Equipamentos par...,BRL,Delayed Quote,R$,None,None,None,942517824
4,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,...,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA,Quote not found for ticker symbol: ICBR3.SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,...,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA
375,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,...,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA
376,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote 

Processing batches of diferent, first 1: historical financial

In [15]:
# # Exporting Excel file Dataframes
# df.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/AtivosBr.xlsx',
#             sheet_name="stocks")

# dfbeta.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Beta.xlsx'
#             ,sheet_name="key_stats")

# addata.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Price.xlsx',
#             sheet_name="price") 

Local Machine Version

In [16]:
# df.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/AtivosBR.xlsx',
#            sheet_name="stocks",
#            engine='openpyxl')

# dfbeta.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Beta.xlsx',
#                 sheet_name="key_stats")

# addata.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Price.xlsx',
#             sheet_name="price")

# balance_sheet.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Balance.xlsx',
#             sheet_name="balance")

In [17]:
# Define the output directory and filename
output_directory = "../RAW_DATA/tst"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Files that will be created
filenames = ["AtivosBR.xlsx","Beta.xlsx","Financial.xlsx","Price.xlsx"]

# Dataframes of objects
dataframes = [historical_data,historical_financial,historical_balance,historical_finandata]

# Sheet names
sheet_names = ["historical_data","historical_financial","historical_balance","historical_finance_data"]

for item in range(0,len(filenames)):
    # Specify the full file path
    output_file = os.path.join(output_directory, filenames[item])
    #
    dataframes[item].to_excel(excel_writer=output_file, sheet_name=sheet_names[item], engine='openpyxl')


# for filename in filenames:
#     # Specify the full file path
#     output_file = os.path.join(output_directory, filename)
    
#     # This file is ok
#     if filename == "AtivosBR.xlsx":
#         # Save the DataFrame as an Excel file
#         historical_data.to_excel(excel_writer=output_file, sheet_name="historical_data", engine='openpyxl')
    
#     elif filename == "Beta.xlsx":
#         historical_financial.to_excel(excel_writer=output_file, sheet_name="historical_financial", engine='openpyxl')
    
#     elif filename == "Financial.xlsx":
#         historical_balance.to_excel(excel_writer=output_file, sheet_name="historical_balance", engine='openpyxl')
    
#     elif filename == "Price.xlsx":
#         historical_finandata.to_excel(excel_writer=output_file, sheet_name="historical_finance_data", engine='openpyxl')